<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Youtube_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_community langchain_groq youtube_transcript_api faiss-cpu

  Using cached langchain_community-0.3.18-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_groq-0.2.4-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [3]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
import textwrap

In [4]:
embeddings=HuggingFaceEmbeddings()

<ipython-input-4-5897ca6a0cdb>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings()
<ipython-input-4-5897ca6a0cdb>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY'] = userdata.get('groq_api_key')

In [14]:
def create_db_from_youtube_video_url(video_url):
  loader=YoutubeLoader.from_youtube_url(video_url)
  transcript = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=500)
  docs = text_splitter.split_documents(transcript)
  db = FAISS.from_documents(docs, embeddings)
  return db

In [16]:
def get_response_from_query(db, query, k=4):
  docs = db.similarity_search(query, k=k)
  docs_page_content="".join([d.page_content for d in docs])
  chat=ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
  template="""
  You are an assistant for question-answering tasks.
  Use the following pieces of retrieved context to answer the question about youtube video transcript.
  Video Transcript Similarity docs: {docs}
  If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
  """
  system_message_prompt=SystemMessagePromptTemplate.from_template(template)
  human_template="Answer the following question: {question}"
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

  chat_prompt = ChatPromptTemplate.from_messages(
      [system_message_prompt, human_message_prompt]
  )

  chain = LLMChain(llm=chat, prompt = chat_prompt)

  response = chain.run(question=query, docs=docs_page_content)
  response = response.replace("\n","")
  return response, docs

In [17]:
video_url = "https://www.youtube.com/watch?v=g_HhjKQXbcQ"
db=create_db_from_youtube_video_url(video_url)
query = "What is this video about?"
response, docs =get_response_from_query(db, query)
print(textwrap.fill(response, width=70))

<ipython-input-16-6c22fb868c69>:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt = chat_prompt)
<ipython-input-16-6c22fb868c69>:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(question=query, docs=docs_page_content)


This video is about creating an assistant that can answer questions
about YouTube videos using Langchain and the YouTube Transcript API.
The project involves transcribing YouTube videos and creating a
database from the transcript. The video appears to be a tutorial on
building this project, with the creator walking through the steps and
code required to complete it.


In [20]:
print([doc.page_content for doc in docs])

["hi everyone and welcome back to the multivers of 100 plus data science project Series in this video you'll learn how to create a assistant that can answer questions about YouTube videos so without wasting any time let us start our tutoral right now so in to creating this project you're using here the Lang chain and we also installing here some of the important lies like Lang chain Community Lang chain Grog YouTube transcript API as well as the five CPU which is not nothing but one back to isore database and using this YouTube transcript API you can directly accessing the video from the YouTube as well as we can transcribe the video and using this Lang chain gro we can accessing here the gro Cloud API from the gro Cloud workspace and using this community you're going to do some kind of operations in our videos using the rack so let install it by uh clicking this button or you can also using the shift enter so you can say requirement already satisfied so we're going to click on here an